In [1]:
import os
import numpy as np

from tensorflow.keras.applications import NASNetMobile

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_dir = '/Users/saikarthik/Downloads/untitled folder/parkinsons_dataset'


In [3]:
img_height, img_width = 224, 224 
batch_size = 32


In [4]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [5]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

Found 665 images belonging to 2 classes.


In [6]:

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 166 images belonging to 2 classes.


In [7]:
base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

2025-04-02 16:11:33.520999: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-02 16:11:33.521030: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-02 16:11:33.521042: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-02 16:11:33.521075: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-02 16:11:33.521088: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 1ed92395b5b598bdda52abe5c0dbfd63 so we will re-download the data.
19993432/19993432 [==============================] - 7s 0us/step


In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
)

Epoch 1/100


2025-04-02 16:11:48.853216: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.7734 - accuracy: 0.6556

2025-04-02 16:11:58.672349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 19s 584ms/step - loss: 0.7734 - accuracy: 0.6556 - val_loss: 0.6296 - val_accuracy: 0.7349
Epoch 2/100
21/21 [==============================] - 5s 239ms/step - loss: 0.5698 - accuracy: 0.7444 - val_loss: 0.5387 - val_accuracy: 0.7470
Epoch 3/100
21/21 [==============================] - 5s 234ms/step - loss: 0.4941 - accuracy: 0.7684 - val_loss: 0.4960 - val_accuracy: 0.7651
Epoch 4/100
21/21 [==============================] - 5s 234ms/step - loss: 0.4514 - accuracy: 0.7910 - val_loss: 0.4517 - val_accuracy: 0.7831
Epoch 5/100
21/21 [==============================] - 5s 235ms/step - loss: 0.4063 - accuracy: 0.8241 - val_loss: 0.4115 - val_accuracy: 0.7952
Epoch 6/100
21/21 [==============================] - 5s 235ms/step - loss: 0.3787 - accuracy: 0.8421 - val_loss: 0.3832 - val_accuracy: 0.8072
Epoch 7/100
21/21 [==============================] - 5s 235ms/step - loss: 0.3330 - accuracy: 0.8767 - val_loss: 0.3496 - val_accuracy: 0.8193
Epoch 8/10

In [18]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

validation_generator.reset()
predictions = (model.predict(validation_generator) > 0.5).astype("int64")
true_classes = validation_generator.classes

print("\nClassification Report:")
print(classification_report(true_classes, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predictions))

6/6 [==============================] - 1s 162ms/step - loss: 0.1569 - accuracy: 0.9277
Validation Accuracy: 92.77%
6/6 [==============================] - 1s 167ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.80      0.77       122
           1       0.27      0.20      0.23        44

    accuracy                           0.64       166
   macro avg       0.50      0.50      0.50       166
weighted avg       0.61      0.64      0.63       166


Confusion Matrix:
[[98 24]
 [35  9]]
